<a href="https://colab.research.google.com/github/IverMartinsen/ColabNotebooks/blob/main/LoRA_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Fine-tuning with LoRA**
This script is mostly copy-pasted from this: https://huggingface.co/docs/peft/main/en/task_guides/image_classification_lora

**Install dependencies**

In [ ]:
pip install -q transformers accelerate evaluate datasets peft

Import modules

In [ ]:
import torch
import transformers
import accelerate
import peft
import evaluate
import numpy as np
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
)
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)


**Load data (this will take ~5mins).**

For documentation about the Dataset class: https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset

In [ ]:
ds = load_dataset("food101", split="train[:5000]")

**Split into train/test**

In [ ]:
splits = ds.train_test_split(test_size=0.1)

train_ds = splits["train"]
val_ds = splits["test"]

**Construct preprocessing pipelines for training and validation data**

In [ ]:
model_checkpoint = "google/vit-base-patch16-224-in21k"

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)


**Load the pretrained Vision Transformer**

In [ ]:
# create a simple string-to-int dict mapping for labeling the data
labels = ds.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)


**Apply a LoRA wrapper to the model**

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

**Set training arguments**

For documentation about the Trainer class: https://huggingface.co/docs/transformers/main_classes/trainer#api-reference%20][%20transformers.Trainer

In [ ]:
batch_size = 128

args = TrainingArguments(
    "my-finetuned-lora-food101",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    label_names=["labels"],
)


metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)


**Train model**

In [ ]:
train_results = trainer.train()

**Evaluation**

In [ ]:
trainer.evaluate(val_ds)